In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = 'C:/Users/ISHMO_CT/Downloads/my-projects/Titanic_survival/titanic/train.csv'
test_data = 'C:/Users/ISHMO_CT/Downloads/my-projects/Titanic_survival/titanic/test.csv'

In [3]:
def wrangle(df):
    df = pd.read_csv(df)
    df.set_index('PassengerId', inplace=True)
    threshold =100    
    cols_to_drop = [col for col in df.select_dtypes('object') if df[col].nunique() > threshold]
    df.drop(columns=cols_to_drop, inplace=True)
    return df

In [4]:
df = wrangle(train_data)

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S


In [6]:
target = 'Survived'
y = df[target]
X = df.drop(columns=target)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val =train_test_split(X,y, test_size=0.2, random_state=42)

In [8]:
baseline = y_train.value_counts(normalize=True).max()
print(baseline)

0.6235955056179775


In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from category_encoders import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

model_lr = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    LogisticRegression(penalty='l2', max_iter=300)
)

In [10]:
model_lr.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Sex', 'Embarked'],
                                mapping=[{'col': 'Sex', 'data_type': dtype('O'),
                                          'mapping': male      1
female    2
NaN      -2
dtype: int64},
                                         {'col': 'Embarked',
                                          'data_type': dtype('O'),
                                          'mapping': S      1
C      2
Q      3
NaN    4
dtype: int64}])),
                ('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=300))])

In [11]:
# Accurancy score for trainning 
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_train, model_lr.predict(X_train))
print(acc)

0.8019662921348315


In [12]:
# aacurancy score for Validation
acc = accuracy_score(y_val, model_lr.predict(X_val))
print(acc)

0.7988826815642458


In [13]:
# accuramcy for test score
ts = wrangle(test_data)
cols_to_drop_test = ['Cabin',]
ts.drop(columns=cols_to_drop_test, inplace=True)
ts.shape

(418, 7)

In [14]:
predictions = pd.DataFrame(data= model_lr.predict(ts),index = ts.index, columns= ['Survived'])
predictions

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [15]:
predictions.to_csv('submission.csv')

In [16]:
#Decision regression
from sklearn.tree import DecisionTreeClassifier
model_dt = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    DecisionTreeClassifier(random_state=42, max_depth=5)
)

In [17]:
model_dt.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Sex', 'Embarked'],
                                mapping=[{'col': 'Sex', 'data_type': dtype('O'),
                                          'mapping': male      1
female    2
NaN      -2
dtype: int64},
                                         {'col': 'Embarked',
                                          'data_type': dtype('O'),
                                          'mapping': S      1
C      2
Q      3
NaN    4
dtype: int64}])),
                ('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=5, random_state=42))])

In [18]:
acc_model_dt_train = accuracy_score(y_train, model_dt.predict(X_train))
acc_model_dt_val = accuracy_score(y_val, model_dt.predict(X_val))
print(acc_model_dt_train)
print(acc_model_dt_val)

0.851123595505618
0.7988826815642458


In [19]:
model_dt.named_steps['decisiontreeclassifier'].get_depth()

5

In [26]:
depths = range(1, 20)

train_dt_acc = []
val_dt_acc = []

for depth in depths:
    model_tree_ddpth = make_pipeline(
        OrdinalEncoder(),
        SimpleImputer(),
        DecisionTreeClassifier(max_depth=depth, random_state=42)
    )
    model_tree_ddpth.fit(X_train, y_train)
    train_dt_acc.append(model_tree_ddpth.score(X_train, y_train))
    val_dt_acc.append(model_tree_ddpth.score(X_val, y_val))
print(train_dt_acc)
print(val_dt_acc)

[0.7879213483146067, 0.8033707865168539, 0.8342696629213483, 0.8398876404494382, 0.851123595505618, 0.8665730337078652, 0.8820224719101124, 0.898876404494382, 0.9073033707865169, 0.9129213483146067, 0.9283707865168539, 0.9367977528089888, 0.9438202247191011, 0.949438202247191, 0.9550561797752809, 0.9620786516853933, 0.9719101123595506, 0.973314606741573, 0.9775280898876404]
[0.7821229050279329, 0.7653631284916201, 0.7988826815642458, 0.7988826815642458, 0.7988826815642458, 0.8044692737430168, 0.8100558659217877, 0.7988826815642458, 0.7988826815642458, 0.7932960893854749, 0.7932960893854749, 0.7988826815642458, 0.7932960893854749, 0.7877094972067039, 0.7877094972067039, 0.7932960893854749, 0.7932960893854749, 0.7932960893854749, 0.776536312849162]


In [21]:
predictions = pd.DataFrame(data = model_dt.predict(ts), index=ts.index, columns=['Survived'])

In [22]:
predictions.to_csv('submission.csv')

In [23]:
# random forest
from sklearn.ensemble import RandomForestClassifier
model_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_jobs=-1)
)

In [24]:
model_rf.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Sex', 'Embarked'],
                                mapping=[{'col': 'Sex', 'data_type': dtype('O'),
                                          'mapping': male      1
female    2
NaN      -2
dtype: int64},
                                         {'col': 'Embarked',
                                          'data_type': dtype('O'),
                                          'mapping': S      1
C      2
Q      3
NaN    4
dtype: int64}])),
                ('simpleimputer', SimpleImputer()),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, random_state=42))])

In [25]:
acc_model_rf = accuracy_score(y_train, model_rf.predict(X_train))
print(acc_model_rf)

0.9803370786516854
